In [4]:
# Import Dependencies
import pandas as pd
from census import Census
from config import census_key
import cpi # this will be used to determine the cost of everything in today's dollars. cpi.inflate(amount,year of amount)
#install cpi with pip install cpi on command line
cpi.update()
low_memory=False

In [88]:
# Import CSVs 
filepath = "Resources/"
kent_zipcodes = pd.read_csv(filepath+"kent_zipcodes.csv",squeeze=True,header=None)
oakland_zipcodes = pd.read_csv(filepath+"oakland_zipcodes.csv",squeeze=True,header=None)
saginaw_zipcodes = pd.read_csv(filepath+"saginaw_zipcodes.csv",squeeze=True,header=None)



0     49508
1     49504
2     49507
3     49503
4     49341
5     49506
6     49548
7     49505
8     49546
9     49525
10    49418
11    49519
12    49509
13    49315
14    49301
15    49534
16    49316
17    49512
18    49319
19    48838
20    49321
21    49331
22    49345
23    48809
24    49544
25    49306
26    49302
27    49343
28    49330
29    49326
30    49325
31    49351
32    49550
33    49317
34    49355
35    49357
36    49356
37    49501
38    49468
39    49502
40    49510
41    49515
42    49514
43    49518
44    49516
45    49523
46    49528
47    49530
48    49560
49    49555
50    49599
51    49588
Name: 0, dtype: int64

In [94]:
# Get data from API for Census
census_df = pd.DataFrame({"Year":[0],
                          "Zipcode":["0"],
                          "Total Population":[0],
                          "White Population":[0],
                          "Black Population":[0],
                          "Employed Count":[0],
                          "Unemployed Count":[0],
                          "Poverty Count":[0],
                          "Poverty Count - White":[0],
                          "Poverty Count - Black":[0],
                          "Median Income":[0],
                          "Median Rent":[0],
                          "Median Home Value":[0]})
census_df = census_df.astype({"Year":"int64","Zipcode":"object"})
for year in [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]: 
    c = Census(census_key,year) 
    census_data = c.acs5.get(("NAME", "B19013_001E", "B23025_004E", "B23025_005E", "B17001_002E", 
            "B17001A_002E","B17001B_002E", "B01003_001E",
            "B02001_002E", "B02001_003E","B25058_001E", "B25077_001E"
           ), {"for": "zip code tabulation area:*"})
    census_raw_df = pd.DataFrame(census_data)
    year_count = len(census_data)
    year_series = pd.Series([year for x in range(year_count)])    
    census_raw_df["Year"] = year_series
    #Column Names
    census_raw_df = census_raw_df.rename(columns = {
            "B19013_001E" : "Median Income", 
            "B23025_004E" : "Employed Count", 
            "B23025_005E" : "Unemployed Count", 
            "B17001_002E" : "Poverty Count", 
            "B17001A_002E" : "Poverty Count - White" , 
            "B17001B_002E" : "Poverty Count - Black" , 
            "B01003_001E" : "Total Population", 
            "B02001_002E" : "White Population", 
            "B02001_003E" : "Black Population", 
            "B25058_001E" : "Median Rent", 
            "B25077_001E" : "Median Home Value", 
            "NAME" : "Name",
            "zip code tabulation area": "Zipcode"})
    census_raw_df = census_raw_df.astype({"Year":"int64","Zipcode":"object","Median Income":"float","Employed Count":"float","Unemployed Count":"float","Poverty Count":"float",
             "Poverty Count - White":"float", "Poverty Count - Black":"float", "Total Population":"float",
             "White Population":"float","Black Population":"float", "Median Rent":"float","Median Home Value":"float"})
    #test = census_raw_df[~(census_raw_df["Zipcode"].isin(kent_zipcodes | oakland_zipcodes | saginaw_zipcodes))]
#     test=pd.DataFrame(census_raw_df.loc[census_raw_df["state"] == "26",:])
    census_df = census_df.astype({"Zipcode":"object"})
    test = test.astype({"Year":"int64","Zipcode":"object","Median Income":"float","Employed Count":"float","Unemployed Count":"float","Poverty Count":"float",
             "Poverty Count - White":"float", "Poverty Count - Black":"float", "Total Population":"float",
             "White Population":"float","Black Population":"float", "Median Rent":"float","Median Home Value":"float"})
    census_df = pd.merge(census_df,test,how="outer")

 



In [102]:
census_df = census_df.drop(columns = "Name")

,Year,Zipcode,Total Population,White Population,Black Population,Employed Count,Unemployed Count,Poverty Count,Poverty Count - White,Poverty Count - Black,Median Income,Median Rent,Median Home Value,Name
0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
1,2020,29590,3543,718,2617,1104,46,869,177,663,30985,503,68600,ZCTA5 29590
2,2020,93306,74296,51869,1727,28717,2379,15853,10291,796,54450,867,211100,ZCTA5 93306
3,2020,93660,4082,2231,40,1313,229,1057,492,40,39625,597,171700,ZCTA5 93660
4,2020,93110,15777,12042,311,7692,286,1332,943,100,93264,1877,844300,ZCTA5 93110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33116,2020,16623,552,550,0,201,18,80,80,0,51667,621,147900,ZCTA5 16623
33117,2020,16627,2118,1980,9,988,124,324,275,0,45000,392,75900,ZCTA5 16627
33118,2020,16634,315,311,0,127,19,24,24,0,51500,-666666666,97500,ZCTA5 16634
33119,2020,16640,707,701,3,266,12,167,167,0,55982,511,117500,ZCTA5 16640


In [109]:
## This is not working.
kent_df = census_df.query(census_df["Zipcode"].isin(kent_zipcodes))
kent_df

ValueError: expr must be a string to be evaluated, <class 'pandas.core.series.Series'> given